# Aggregatie

Met aggragatie bedoelen we het samenvatten van de gegevens van een aantal documenten in een enkele waarde
Voorbeelden van aggregatie-operatoren zijn: `sum` (en daarmee ook `count`), `avg`, `min`, `max`.
(Dit zijn ook typische aggregatie-functies in spreadsheets.)
Dit samenvatten combineren we vaak met *groepering*: we willen de samenvattende waarde per groep weten.

> In SQL heb je voor aggregatie dezelfde aggrgatiefuncties zoals hierboven genoemd; 
  deze gebruik je in het SELECT-deel van de query. Voor groepering gebruik je GROUP BY.

Voor meer informatie over aggregatie in MongoDB, zie: https://docs.mongodb.com/manual/aggregation

Voor de aggregatie-voorbeelden gebruiken we naast de contacten-collection, ook een collection met agenda-items.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]

mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

contacts = db.contacts
contacts.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

In [ ]:
agenda = db.agenda
agenda.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c agenda agenda.json')

## Aggregatie in MongoDB

De `aggregate`-functie heeft een lijst van parameters (`[...]`).
De belangrijkste twee parameters zijn objecten met de volgende velden (properties):

1. `$match`: het *filter* voor de documenten in de aggregatie (als in `collection.find`, vgl. SQL `WHERE`)
2. `$group`: het veld voor de groepering (`_id`), en het veld voor de aggregatie met de vorm van de aggregatie (count, sum, enz.)

Zie ook de figuur in: https://docs.mongodb.com/manual/aggregation/#aggregation

Merk op dat je voor de veldnamen van het oorspronkelijke document **in het $group-deel een `$` schrijft.** Bijvoorbeeld: `$address.city`.

### Voorbeeld 1

In het eerste voorbeeld tellen we het aantal contacten per plaats.
We laten het `$match` query-document hier weg: we onderzoeken alle documenten in de collection.
We groeperen per plaats (via de `_id`) en geven per plaats het aantal (`count`) dat we berekenen door 1 te sommeren voor alle documenten (MongoDB heeft geen aparte `count`-functie).

In [ ]:
cursor = contacts.aggregate([{"$group":{"_id": "$address.city", "count": {"$sum": 1} }}])
list(cursor)

### Voorbeeld 2

Voor het tweede voorbeeld gebruiken we de agenda-items.
Bekijk eerst het bestand [agenda.json](agenda.json) om een indruk te krijgen van de gegevens in deze items.

We willen weten hoeveel tijd we voor de verschillende onderwerpen (`subject`) in de agenda hebben staan.
Merk op dat voor de veldnamen in het $group-deel een `$` geschreven is: `$subject` en `$duration`.


In [ ]:
cursor = agenda.aggregate([{"$group":{"_id": "$subject", "count": {"$sum": "$duration"} }}])
list(cursor)

### Opdracht

Maak een query voor het bepalen van de totale tijd per vergaderlocatie.

### Voorbeeld 3

We gebruiken hier het `$match`-deel met een filter-document: we willen alleen de documenten waarvan de `participants` de persoon met email-adres: hdb@example.com bevat.
Merk op dat dit een *normaal* filter-document is, als bij `collection.find`: de namen van de velden (properties) schrijf je hier zonder `$`.

De totale vergadertijd per locatie met de persoon met email-adres: hdb@example.com:

In [ ]:
cursor = agenda.aggregate([{"$match": {"participants": { "email": "hdb@example.com"}}}, 
                           {"$group": {"_id": "$location", "count": {"$sum": "$duration"}}} ])
list(cursor)

### Opdracht

Maak een query voor het bepalen van de totale tijd per vergaderlocatie, voor het onderwerp (subject) "Beleidsplan".
